In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import re

In [2]:
data = pd.read_csv('Twitter_Data.csv')


In [3]:
data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


In [4]:
#comment=data['clean_text'].astype(str)
comment = list(data['clean_text'])
sent = list(data['category'])
comment_train, comment_test, sent_train, sent_test = train_test_split(comment, sent, test_size=0.2)

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
nltk.download('stopwords')
en_stopwords = set(stopwords.words('english'))
#ps = PorterStemmer()
ps = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def getCleanComment(review):
    review = str(review)
    review = review.lower()
    review = review.replace('"','')
    review = review.replace(';','')
    review = review.replace('-','')
    review = review.replace(',','')
    review = re.sub('\d', '', review)
    tokens = tokenizer.tokenize(review)
    new_tokens = [i for i in tokens if i not in en_stopwords]
    stemmed_tokens = [ps.stem(i) for i in new_tokens]
    
    cleaned_review = ' '.join(stemmed_tokens)
    return cleaned_review

In [8]:
comment_train = [getCleanComment(i) for i in comment_train]

In [9]:
#comment_train

In [10]:
cv = CountVectorizer()
comment_train_vec = cv.fit_transform(comment_train)
print(comment_train_vec.shape)
#print(cv.get_feature_names())

(130378, 75181)


In [11]:
mnb = MultinomialNB()
mnb.fit(comment_train_vec,sent_train)

MultinomialNB()

### Testing

In [12]:
comment_test = [getCleanComment(i) for i in comment_test]

In [13]:
comment_test_vec = cv.transform(comment_test)
print(comment_test_vec.shape)

(32595, 75181)


In [14]:
sentiment_prediction = mnb.predict(comment_test_vec)

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
accuracy_score(sent_test, sentiment_prediction)

0.7190059825126554

## Bernaulli NB

In [17]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(comment_train_vec,sent_train)

BernoulliNB()

In [18]:
sentiment_prediction_bnb = bnb.predict(comment_test_vec)
accuracy_score(sent_test, sentiment_prediction_bnb)

0.745482435956435

In [ ]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC(dual=False)
lsvm.fit(comment_train_vec,sent_train)

In [ ]:
sentiment_prediction_svm = lsvm.predict(comment_test_vec)
accuracy_score(sent_test, sentiment_prediction_svm)